In [1]:
# from d2l import torch as d2l
import torch
from torch import nn
from torch.nn import functional as F
from torch import nn, optim

import sys
sys.path.append("..")
# import d2lzh_pytorch.torch as d2l

from d2lzh_pytorch.cnn.load_data.load_hotdog import get_hotdog

from d2lzh_pytorch.cnn.model.pretrain_resnet18 import get_pretrain_resnet18

from d2lzh_pytorch.cnn.train.train_fine_tuning import train_fine_tuning

from d2lzh_pytorch.myUtils import try_gpu



print(torch.__version__)

1.13.1+cu117


In [2]:
train_iter, test_iter = get_hotdog()

In [3]:

pretrained_net = get_pretrain_resnet18()

/home/mylady/.virtualenvs/dl-pytorch/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mylady/.virtualenvs/dl-pytorch/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
output_params = list(map(id, pretrained_net.fc.parameters()))

feature_params = filter(lambda p: id(p) not in output_params, pretrained_net.parameters())


lr = 0.01

optimizer = optim.SGD([{'params': feature_params},
                       {'params': pretrained_net.fc.parameters(), 'lr': lr * 10}],
                       lr=lr, weight_decay=0.001)

In [6]:

# 开始微调
train_fine_tuning(pretrained_net, 
                  train_iter, test_iter,
                  optimizer, 
                  batch_size=128, num_epochs=5)

training on  cuda
epoch 1, loss 5.8965, train acc 0.526, test acc 0.864, time 10.4 sec
epoch 2, loss 0.2199, train acc 0.894, test acc 0.846, time 9.4 sec
epoch 3, loss 0.0849, train acc 0.918, test acc 0.926, time 9.4 sec
epoch 4, loss 0.1177, train acc 0.880, test acc 0.911, time 9.4 sec
epoch 5, loss 0.0505, train acc 0.925, test acc 0.916, time 9.4 sec


In [8]:
# 保存模型
net = pretrained_net.to("cpu")

import datetime
import os
str_time = str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
save_path = 'pretrain_RestNet18_hotdog_%s.pt' % str_time
torch.save(net, save_path)  # 全保存
print("训练完毕, 模型 %s 已保存至当前路径" % save_path)
print("模型大小是: %0.2f M" % (os.path.getsize(save_path) / 1024 /1024))

训练完毕, 模型 pretrain_RestNet18_hotdog_2023-07-02_01-24-45.pt 已保存至当前路径
模型大小是: 42.73 M


In [9]:
from torchsummary import summary

summary(net)


Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            9,408
├─BatchNorm2d: 1-2                       128
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
|    └─BasicBlock: 2-1                   --
|    |    └─Conv2d: 3-1                  36,864
|    |    └─BatchNorm2d: 3-2             128
|    |    └─ReLU: 3-3                    --
|    |    └─Conv2d: 3-4                  36,864
|    |    └─BatchNorm2d: 3-5             128
|    └─BasicBlock: 2-2                   --
|    |    └─Conv2d: 3-6                  36,864
|    |    └─BatchNorm2d: 3-7             128
|    |    └─ReLU: 3-8                    --
|    |    └─Conv2d: 3-9                  36,864
|    |    └─BatchNorm2d: 3-10            128
├─Sequential: 1-6                        --
|    └─BasicBlock: 2-3                   --
|    |    └─Conv2d: 3-11                 73,728
|    |    └─BatchNorm2d: 3-12            25

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            9,408
├─BatchNorm2d: 1-2                       128
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
|    └─BasicBlock: 2-1                   --
|    |    └─Conv2d: 3-1                  36,864
|    |    └─BatchNorm2d: 3-2             128
|    |    └─ReLU: 3-3                    --
|    |    └─Conv2d: 3-4                  36,864
|    |    └─BatchNorm2d: 3-5             128
|    └─BasicBlock: 2-2                   --
|    |    └─Conv2d: 3-6                  36,864
|    |    └─BatchNorm2d: 3-7             128
|    |    └─ReLU: 3-8                    --
|    |    └─Conv2d: 3-9                  36,864
|    |    └─BatchNorm2d: 3-10            128
├─Sequential: 1-6                        --
|    └─BasicBlock: 2-3                   --
|    |    └─Conv2d: 3-11                 73,728
|    |    └─BatchNorm2d: 3-12            25

In [10]:
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  